# Лабораторная работа 1

### Разделение массива на количественный и качественный

In [2]:
# импорт библиотек
import numpy as np
import pandas as pd
import scipy.stats as stats

In [3]:
# выгрузка данных
df_0 = pd.read_csv('V11.csv', sep=';', decimal=',')
df_0.head()

,A6,A10,A13,A16,A18,A20,A21,A22,A23,A37,A40
0,key,coated,line,821,tabloid,1911,25.0,38,0.200,40,noband
1,key,coated,line,816,spiegel,1910,45.0,54,0.167,35,noband
2,key,coated,line,827,tabloid,1911,50.0,46,0.200,40,noband
3,key,uncoated,line,802,catalog,1910,47.5,52,0.267,33,band
4,key,uncoated,line,815,catalog,1911,50.0,60,0.300,40,band


In [4]:
# разделим на 2 датафрейма - качественные и количественные данные
df_1 = df_0.iloc[:, 0:6]
df_1['A40'] = df_0['A40']
#df_1.head()

df_2 = df_0.iloc[:, 6:10]
df_2['A40'] = df_0['A40']
df_2.head()

,A21,A22,A23,A37,A40
0,25.0,38,0.200,40,noband
1,45.0,54,0.167,35,noband
2,50.0,46,0.200,40,noband
3,47.5,52,0.267,33,band
4,50.0,60,0.300,40,band


In [5]:
# Произведем калибровку качественных данных с учетом априорного шанса с поправкой Лапласа
# С помощью формулы мы получили числа в таблице Excel, вставим их в дф, изменив старые значения

# A6
replace_dict = {'key': 0.001311808, 'type': 1}
df_1['A6'] = df_1['A6'].replace(replace_dict)

# A10
replace_dict = {'coated': 0.00200664, 'uncoated': 0.002898667, 'super': 1}
df_1['A10'] = df_1['A10'].replace(replace_dict)

# A13
replace_dict = {'line': 0.001308837, 'xylol': 0.176470588, 'lactol': 1, 'naptha': 0.076923077, 'other': 1}
df_1['A13'] = df_1['A13'].replace(replace_dict)

# A16
replace_dict = {821: 0.021956088, 802: 0.007407407, 813: 0.005544006, 
                824: 0.003076923, 815: 0.016248154, 816: 0.00752909, 
                827: 0.009784736, 828: 0.011124845
}
df_1['A16'] = df_1['A16'].replace(replace_dict)

# A18
replace_dict = {'tabloid': 0.00255052, 'spiegel': 0.009983361, 'catalog': 0.003649231}
df_1['A18'] = df_1['A18'].replace(replace_dict)

# A20
replace_dict = {1911: 0.004417834, 1910: 0.001846365, 'other': 1}
df_1['A20'] = df_1['A20'].replace(replace_dict)

df_1.head()

C:\Users\Екатерина\AppData\Local\Temp\ipykernel_22380\914489761.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_1['A6'] = df_1['A6'].replace(replace_dict)
C:\Users\Екатерина\AppData\Local\Temp\ipykernel_22380\914489761.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_1['A10'] = df_1['A10'].replace(replace_dict)
C:\Users\Екатерина\AppData\Local\Temp\ipykernel_22380\914489761.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explic

,A6,A10,A13,A16,A18,A20,A40
0,0.001312,0.002007,0.001309,0.021956,0.002551,0.004418,noband
1,0.001312,0.002007,0.001309,0.007529,0.009983,0.001846,noband
2,0.001312,0.002007,0.001309,0.009785,0.002551,0.004418,noband
3,0.001312,0.002899,0.001309,0.007407,0.003649,0.001846,band
4,0.001312,0.002899,0.001309,0.016248,0.003649,0.004418,band


In [6]:
# Проведем дискретизацию с равной частотой количественных данных
quantiles = [0, 0.2, 0.4, 0.6, 0.8, 1]

for column in df_2:
    if column != 'A40':

        # Получение уникальных квантилей
        q = df_2[column].quantile(quantiles)
        unique_quantiles = q.unique()
        print(unique_quantiles)

        # этот массив потом вставить можно в дф
        arr = []

        # сравнение каждого значения дф с границами инт-лов
        for value in df_2[column]:
            for i in range(len(unique_quantiles)):
                if i == 0 and unique_quantiles[i] <= value <= unique_quantiles[i+1]:
                    arr.append(i+1)
                    break
                if unique_quantiles[i] < value <= unique_quantiles[i+1]:
                    arr.append(i+1)

        # создание нового столбца
        df_2[f'{column}_dis'] = arr
        

# Вывод результата
print(df_2)



[25.  37.5 40.  47.  50.  72.5]
[35. 43. 48. 54. 58. 72.]
[0.133 0.2   0.267 0.3   0.333 0.533]
[30. 35. 40. 45.]
      A21  A22    A23  A37     A40  A21_dis  A22_dis  A23_dis  A37_dis
0    25.0   38  0.200   40  noband        1        1        1        2
1    45.0   54  0.167   35  noband        3        3        1        1
2    50.0   46  0.200   40  noband        4        2        1        2
3    47.5   52  0.267   33    band        4        3        2        1
4    50.0   60  0.300   40    band        4        5        3        2
..    ...  ...    ...  ...     ...      ...      ...      ...      ...
495  30.0   54  0.500   35  noband        1        3        5        1
496  52.5   62  0.433   35    band        5        5        5        1
497  52.0   40  0.300   40  noband        5        1        3        2
498  60.0   56  0.300   40    band        5        4        3        2
499  45.0   53  0.300   35  noband        3        3        3        1

[500 rows x 9 columns]


In [7]:
# преобразуем A40 в столбец с 0 и 1 в обоих дф
df_1['A40'] = df_1['A40'].apply(lambda x: 1 if x == 'band' else 0)
df_2['A40'] = df_2['A40'].apply(lambda x: 1 if x == 'band' else 0)
print(df_1)
print(df_2)

           A6       A10       A13       A16       A18       A20  A40
0    0.001312  0.002007  0.001309  0.021956  0.002551  0.004418    0
1    0.001312  0.002007  0.001309  0.007529  0.009983  0.001846    0
2    0.001312  0.002007  0.001309  0.009785  0.002551  0.004418    0
3    0.001312  0.002899  0.001309  0.007407  0.003649  0.001846    1
4    0.001312  0.002899  0.001309  0.016248  0.003649  0.004418    1
..        ...       ...       ...       ...       ...       ...  ...
495  0.001312  0.002007  0.001309  0.003077  0.002551  0.001846    0
496  0.001312  0.002899  0.001309  0.021956  0.002551  0.001846    1
497  0.001312  0.002899  0.001309  0.007529  0.003649  0.001846    0
498  0.001312  0.002899  0.176471  0.007529  0.003649  0.004418    1
499  0.001312  0.002007  0.001309  0.007529  0.002551  0.001846    0

[500 rows x 7 columns]
      A21  A22    A23  A37  A40  A21_dis  A22_dis  A23_dis  A37_dis
0    25.0   38  0.200   40    0        1        1        1        2
1    45.0   

### Вычисление взаимной информации, коэфициента неопределенности и симметричного коэфициента

In [8]:
# импорт модулей
from sklearn.metrics import mutual_info_score
from scipy.stats import entropy
from sklearn.feature_selection import mutual_info_classif

In [10]:
import math

In [25]:
# создание дф для MI
columns = list(df_1.columns)
df_mi = pd.DataFrame(np.zeros((len(columns), len(columns))), index=columns, columns=columns)

# создаем массив с уникальными значениями из каждого столбца
uniq = []
for column in columns:
    uniq.append(list(df_1[column].unique()))

# поехали рассчитывать MI для каждых двух столбцов
for x_i in range(len(uniq)):
    col1 = columns[x_i]
    for y_i in range(len(uniq)):
        col2 = columns[y_i]

        if col1 != col2:
            # это собственно показатель взаимосвязи
            mi = 0

            # пошла обработка каждого столбца
            for y in uniq[y_i]:
                for x in uniq[x_i]:
                    # считаем вероятность встречи каждого значения из 1 и 2 столбцовЮ иначе говоря di* и dij
                    py = len(df_1[df_1[col2] == y]) / 500
                    px = len(df_1[df_1[col1] == x]) / 500
                    
                    # nij
                    pxy = len(df_1[(df_1[col2] == y) &
                                (df_1[col1] == x)]) / 500
                    
                    # Если nij != 0, то считаем MI согласно формуле (4) из лекции 2
                    if pxy != 0:
                        mi += pxy * math.log2((pxy) / (px * py))
            df_mi.loc[col1, col2] = mi
        else:
            df_mi.loc[col1, col2] = None

# кайфарики
print(df_mi)

      A6       A10       A13       A16       A18       A20       A40
A6   NaN  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
A10  0.0       NaN  0.028193  0.208490  0.185715  0.004514  0.027052
A13  0.0  0.028193       NaN  0.056286  0.029742  0.011425  0.017369
A16  0.0  0.208490  0.056286       NaN  0.664340  0.046477  0.091931
A18  0.0  0.185715  0.029742  0.664340       NaN  0.006847  0.002080
A20  0.0  0.004514  0.011425  0.046477  0.006847       NaN  0.002237
A40  0.0  0.027052  0.017369  0.091931  0.002080  0.002237       NaN


In [30]:
# расчет коэф-та неопределенности и симметричный коэф-т

df_uc = pd.DataFrame(np.zeros((len(columns), len(columns))), index=columns, columns=columns)
df_su = pd.DataFrame(np.zeros((len(columns), len(columns))), index=columns, columns=columns)

for x_i in range(len(uniq)):
    col1 = columns[x_i]
    for y_i in range(len(uniq)):
        col2 = columns[y_i]

        if col1 != col2:
            # коэф-т неопр-ти
            #uc = df_mi[col1][col2] / df_mi[col2][col2]
            
            H_y = 0
            for y in uniq[y_i]:
                # по формуле из лекции считаем энтропию отклика H(этта)
                py = len(df_1[df_1[col2] == y]) / 500                   
                H_y += py * math.log2(py)
            if H_y != 0:
                df_uc.loc[col1, col2] = df_mi[col1][col2] / -H_y

            # симметричный коэф-т
            #su = 2 * df_mi[col1][col2] / (df_mi[col1][col1] + df_mi[col2][col2])

            H_x = 0
            for x in uniq[x_i]:
                # по формуле из лекции считаем энтропию отклика H(кси)
                px = len(df_1[df_1[col1] == x]) / 500                    
                H_x += px * math.log2(px)
            df_su.loc[col1, col2] = df_mi[col1][col2] / (-H_x - H_y)

        else:
            df_uc.loc[col1, col2] = None
            df_su.loc[col1, col2] = None

print(df_uc, "\n", df_su)

      A6       A10       A13       A16       A18       A20       A40
A6   NaN  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
A10  0.0       NaN  0.118992  0.071683  0.139139  0.004851  0.027930
A13  0.0  0.029261       NaN  0.019352  0.022283  0.012279  0.017933
A16  0.0  0.216388  0.237565       NaN  0.497729  0.049951  0.094915
A18  0.0  0.192751  0.125530  0.228413       NaN  0.007359  0.002148
A20  0.0  0.004685  0.048220  0.015980  0.005130       NaN  0.002310
A40  0.0  0.028077  0.073310  0.031608  0.001558  0.002404       NaN 
       A6       A10       A13       A16       A18       A20       A40
A6   NaN  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
A10  0.0       NaN  0.023486  0.053845  0.080807  0.002383  0.014002
A13  0.0  0.023486       NaN  0.017894  0.018924  0.009787  0.014409
A16  0.0  0.053845  0.017894       NaN  0.156564  0.012107  0.023712
A18  0.0  0.080807  0.018924  0.156564       NaN  0.003023  0.000903
A20  0.0  0.002383  0.009787  0.